<br /><h1 style="font-family:Impact,Arial;font-size:70px;">Air seen from desk compared to governments</h1>
<h2 style="font-family:Arial;">Marco Foo</h2>
<h2 style="font-family:Arial;">98044163</h2>
<p><small> 37373 Programming for Data Analysis</small></p>
<p><small><font color=MediumVioletRed>Autumn 2020</font></small>
</p>
<hr style="height:5px;border:none;color:#333;background-color:#333;" />

# Abstract

Air quality  

Write a short (no more than 10 lines) summary of your project's topic and main goals.

<hr style="height:5px;border:none;color:#333;background-color:#333;" />
<h1>1. Motivation</h1>

Write a detailed description of the problem and the original goal(s) of the study. Explain why you were interested in this topic and the type of insight you were hoping to get from the data.

Wanting to do something with the data that my air quality indicator gathers on a daily basis. I'm interested to see how the air in my day-to-day case of nearby a computer inside a home differs to generic readings of what the air quality is told to be outside.

<hr style="height:5px;border:none;color:#333;background-color:#333;"/>
<h1>2. The data</h1>
A clear description of the dataset(s) origin and a discussion of why it was chosen, as well as any restriction regarding the use, dissemination or modification of the data.
Note that wherever possible you should provide a link to the <b>unmodified, raw</b> data set.


My dataset is gathered within non-scientifically through the use of a Kaiterra Laser egg model NB500 placed on my workdesk in an open room.

[insert floorplan here]

The initial proposed dataset was mistakenly assumed to have the full recorded 894 days worth of air quality data recorded, however it was later found out to only contain 10000 records. Such that a bunch of datasets had to be downloaded to get the full recorded air quality data. 

What's recorded my datasets:

"Time Point" - Date format of 'YYYY/MM/DD HH:MM'

"PM10 (µg/m³)" - Integer recording of PM10

"PM2.5 (µg/mÂ³)" - Integer recording of PM2.5

"humidity (%)" - Float recording of the humidity to 2 decimal places

"temp (C)" - Float recording of the temperature to 2 decimal places

"Overall Index (US)" - AQI recording according to the US standard

<p>
The openly avaliable datasets I used to compare with my dataset are found within NSW Govt website with varying options of data output, <a href="https://www.dpie.nsw.gov.au/air-quality/search-for-and-download-air-quality-data">here</a>.
</p>
Specifically, I choose to output multiple differing datasets to draw comparisons with my original datasets.

Hourly AQI averages for the date range of 2019 January 1st to 2020 June 15th for both Prospect and North Parramatta site indexes. 
$#@$#$#for all of the possible sydney-north-west data collection sites. Vineyard's data is all empty, so it's excluded. 
Note, this AQI 

Hourly reported PM2.5, PM10, Windspeed, wind direction, air temperature, relative humidity, global solar radiation and rainfall for the date range of 2019 January 1st to 2020 June 15th for Prospect, and North Parramatta individually. They're seperate datasets due to the query not allowing the combination due to it's size. 
$#$#$for all possible sydney-north-west data collection sites excluding the empty Vineyard option, individually.
<p>
This brings the question of the differing AQI measurements, and whether the recorded AQI is using the US standard or another. 
The <a href="https://www.environment.nsw.gov.au/topics/air/understanding-air-quality-data/air-quality-index">overview of the Australian AQI</a>,
<a href="https://aqicn.org/faq/2014-09-06/australian-air-quality-comparison-with-the-us-epa-aqi-scale/">US and Australian AQI comparison</a> and <a href ="https://www.legislation.gov.au/Details/F2016C00215">government legislation</a> helped with discerning each of the scales. An unsolved problem I encountered researching the National Environment Protection Measure (NEPM) standard, i.e. the standard used in NSW, is that I couldn't find any hard formula other than ranges. As the US AQI standard is widely known and avaliable, as seen in <a href ="https://forum.airnowtech.org/t/the-aqi-equation/169">here</a>. If down, image in the appendix.
</p>


<hr style="height:5px;border:none;color:#333;background-color:#333;" />
<h1>3. Data preparation</h1>

Code and comments on how the raw data is put in a usable form (i.e., how you go from the raw data set(s) to the dataframe you are using for the analysis).

<b>Create as many code, markdown and raw cells as needed</b>

In [241]:
#Import Libraries
import pandas as pd
import numpy as np
import matplotlib as mpt
import seaborn as sns
%matplotlib inline
from pathlib import Path
import os

In [242]:
#MyDatasets
#Archaic dataset names as seen in the MyData_org zip was later manually changed so that it follows the scheme of 'YYYYMMDD HHMMtoYYYYMMDD HHMM' denoting the earliest to latest point of data gathered
mPath = Path('../myData')
list_my_df_fullnames = [item.name for item in sorted(mPath.glob('*.csv'))]
list_my_df = []
for file in list_my_df_fullnames:
    path = Path(mPath, file)
    df_myData = pd.read_csv(path, parse_dates=True, infer_datetime_format=True, header=7, names=['Time', 'PM10', 'PM2.5', 'Humidity', 'Temp', 'USAQI', 'HighestPMType'])
    list_my_df.append(df_myData)
df_allmyData = pd.concat(list_my_df, ignore_index=True)
df_allmyData.head()

,Time,PM10,PM2.5,Humidity,Temp,USAQI,HighestPMType
0,2018-11-15 23:22:00,2.0,2.0,24.44,28.99,8.0,PM2.5
1,2018-11-15 23:30:00,3.0,3.0,25.33,28.02,13.0,PM2.5
2,2018-11-15 23:31:00,4.0,4.0,25.50,27.93,17.0,PM2.5
3,2018-11-15 23:32:00,3.0,1.0,26.08,27.88,4.0,PM2.5
4,2018-11-15 23:33:00,3.0,3.0,25.99,27.85,13.0,PM2.5


In [243]:
# We've got 68 minutes worth of unrecorded air quality statistics, time to find those rows and take them out. As well as the duplicated data that was a consequence of how I had to gather the data from Kaiterra's servers.

print(df_allmyData.count())
print(df_allmyData[9760:9833])

# Looks good, time for the data types and go for rolling averages per hour, similar to the NSW dataset

df_allmyData = df_allmyData.drop_duplicates()
df_allmyData = df_allmyData.dropna()
print(df_allmyData.count())
print(df_allmyData[9760:9833])
print(df_allmyData.dtypes)

Time             336460
PM10             336392
PM2.5            336392
Humidity         336460
Temp             336460
USAQI            336392
HighestPMType    336392
dtype: int64
                     Time  PM10  PM2.5  Humidity   Temp  USAQI HighestPMType
9760  2019-01-02 18:35:00  68.0   58.0     24.49  22.89  152.0         PM2.5
9761  2019-01-02 18:36:00  66.0   57.0     24.50  22.90  152.0         PM2.5
9762  2019-01-02 18:37:00  67.0   59.0     24.39  22.91  153.0         PM2.5
9763  2019-01-02 18:38:00  61.0   55.0     24.34  22.92  149.0         PM2.5
9764  2019-01-02 18:39:00  68.0   60.0     24.33  22.93  153.0         PM2.5
...                   ...   ...    ...       ...    ...    ...           ...
9828  2019-01-02 18:35:00  68.0   58.0     24.49  22.89  152.0         PM2.5
9829  2019-01-02 18:36:00  66.0   57.0     24.50  22.90  152.0         PM2.5
9830  2019-01-02 18:37:00  67.0   59.0     24.39  22.91  153.0         PM2.5
9831  2019-01-02 18:38:00  61.0   55.0     24.34 

In [251]:
# list_my_df_names_early = [item[0:13] for item in list_my_df_fullnames]
# list_my_df_names_latest = [item[17:30] for item in list_my_df_fullnames]
# rMy_df = pd.read_csv("../myData/20181115 2322 to 20190102 1847.csv", parse_dates=True, infer_datetime_format=True, header=7)
# list_my_df[1].head()
# 'Time Point', 'PM10 (µg/m³)', 'PM2.5 (µg/m³)', 'humidity (%)', 'temp (C)', 'Overall Index (US)', 'Primary Pollutant'
# 'Time', 'PM10', 'PM2.5', 'Humidity', 'Temp', 'USAQI', 'HighestPMType'
# my_df_header
# Not Working right annoyingly #my_df_header = str(pd.read_csv("../myData/20181115 2322 to 20190102 1847.csv", header=7, nrows=0).columns)

# Converting types
df_allmyData['Time'] = pd.to_datetime(df_allmyData['Time'], infer_datetime_format=True)
# df_allmyData['PM10'] = df_allmyData['PM10'].astype('int64')
# df_allmyData['PM2.5'] = df_allmyData['PM2.5'].astype('int64')
# df_allmyData['USAQI'] = df_allmyData['USAQI'].astype('int64')
df_allmyData['HighestPMType'] = df_allmyData['HighestPMType'].astype('category')
# print(df_allmyData.dtypes)
# df_allmyData.head()


In [248]:
# df_avgmyData = pd.DataFrame(index=df_allmyData['Time'])

# df_avgmyData['PM10'] = (
#     df_allmyData
#     .assign(PM10=df_allmyData.rolling(window='60m', on=df_allmyData['Time'])['PM10'].mean())
#     .groupby(df_allmyData['Time'].dt.date)['PM10']
#     .last())
# df_avgmyData.head()
# df_allmyData
print(df_allmyData.PM10.rolling(window='60m', on=df_allmyData['Time'])['PM10'].mean())

ValueError: invalid on specified as 0        2018-11-15 23:22:00
1        2018-11-15 23:30:00
2        2018-11-15 23:31:00
3        2018-11-15 23:32:00
4        2018-11-15 23:33:00
                 ...        
336455   2020-06-14 18:40:32
336456   2020-06-14 18:41:32
336457   2020-06-14 18:42:32
336458   2020-06-14 18:43:32
336459   2020-06-14 18:44:32
Name: Time, Length: 292623, dtype: datetime64[ns], must be a column (of DataFrame), an Index or None

In [89]:
df_allmyData[0:10].dtypes

Time Point            datetime64[ns]
PM10 (µg/m³)                 float64
PM2.5 (µg/m³)                float64
humidity (%)                 float64
temp (C)                     float64
Overall Index (US)           float64
Primary Pollutant             object
dtype: object

In [8]:
# Public avaliable datasets
# rParra_df = pd.read_excel("../govtData/nparra.xlsx", 'worksheet1')
# rProspect_df = pd.read_excel("../govtData/prospect.xlsx", 'worksheet1')
rParra_df = pd.read_excel("../govtData/nparra_org.xls", 'worksheet1')
rProspect_df = pd.read_excel("../govtData/prospect_org.xls", 'worksheet1')

<hr style="height:5px;border:none;color:#333;background-color:#333;" />
<h1>4. Exploratory data analysis</h1>

This is the main part of the project. Include code, plots, and detailed explanation of your analysis of the data. Be sure to include enough detail so that anyone can follow and understand what you are doing.

<b>Create as many code, markdown and raw cells as needed</b>

<h2>Questions to ask</h2>
<li> What's a week look like? Maybe heatmaps? </li>
<li> How does each of the PM stats compare to eachother? </li>
<li> How does the meterological stats compare to eachother? </li>
<li> Does wind, rain or other meterological measures have a correlation with the air quality indicators? </li>

<hr style="height:5px;border:none;color:#333;background-color:#333;" />
<h1>5. Results/Insights</h1>

Discuss any non-trivial result or insight into the problem stemming from your analysis.

<b>Create as many code, markdown and raw cells as needed</b>

<hr style="height:5px;border:none;color:#333;background-color:#333;" />
<h1>6. In hindsight...</h1>

In this section you should reflect on your work and what you've learned. In particular you should include:

<ul>
    <li> a discussion of the most challenging aspect of the data analysis and if/how you overcame the challenge.</li>
    <li> a discussion of any programming technique and/or reference you found particularly useful or helpful for your project.</li>
    <li> an assessment of the outcome of the study in relation to the original goals (i.e., have the goals been reached, or did you have to reassess the goals during the project, and if so why?).</li>
    <li> a discussion of what new, or surprising knowledge or insight into the problem you've gained by conducting the study.</li>
    <li> a discussion of whether you would approach the problem differently if given an opportunity to redo the work, and any advice they would give to someone who would want to work on a similar problem.</li>
    <li> a discussion of how your analysis could be improved or extended in future work.</li>

</ul>

In hindsight, the time spent manually changing the filenames was wasted due to how data was read. 

<hr style="height:5px;border:none;color:#333;background-color:#333;" />
<h1>Appendix</h1>
<p> Image of US AQI calculation </p>
<a href="https://forum.airnowtech.org/t/the-aqi-equation/169"><img src="https://puu.sh/FWnlz/f37ea1e65d.png"></a>

The adventure into the air quality data I've gathered. 
My non-scientific and possible real-world-case approach to how I went through the analysais of my gathered dataset.

For context, this dataset is gathered by a Kaiterra LaserEgg model LE-200 that is placed on my work desk at my home. Not in scientific conditions, as the layout of my house can describe this [Insert quick layout]

 